In [12]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
import time


In [13]:
url = "https://www.flipkart.com/"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

In [14]:
categories = {
    "Sports, Fitness, and Outdoor": ["Yoga Accessories", "Protective Gear"]
}
d = {"Product":[],"Price":[],"Original Price":[],"Discount":[],"Rating":[],"Rating & reviews":[], "Category": [], "Sub category": []}

"Grocery and Essentials": ["Snacks", "Beverages", "Cooking Essentials", "Household Supplies", "Dairy Products", "Packaged Foods", "Personal Care", "Pet Supplies"],
"Books and Media": ["Fiction", "Non-fiction", "Academic Books", "Self-help", "Comics", "Magazines", "Music CDs", "Movies", "Children's Books"],
"Sports, Fitness, and Outdoor": ["Sports Equipment", "Fitness Gear", "Outdoor Gear", "Cycling", "Camping & Hiking", "Water Sports", "Yoga Accessories", "Protective Gear"],
"Appliances": ["Washing Machines", "Kitchen Appliances", "Personal Care Appliances"],
"Automotive": ["Car Accessories", "Bike Accessories", "Helmets", "Car Electronics", "Cleaning Supplies", "Tyres", "Oils & Lubricants", "Car Care Kits"]

In [15]:
wait = WebDriverWait(driver, 10)

In [16]:
def scrap_current_page(c,s):
    try:
        elem = driver.find_elements(By.XPATH, "//div[@class='slAVV4']")   
        for i in range(len(elem)):
            d["Category"].append(c)
            d["Sub category"].append(s)
            try:
                d['Product'].append(elem[i].find_element(By.CLASS_NAME,"wjcEIp").text)
            except:
                d['Product'].append(np.nan)
            try:
                d['Price'].append(elem[i].find_element(By.CLASS_NAME,"Nx9bqj").text)
            except:
                d["Price"].append(np.nan)
            try:
                d['Original Price'].append(elem[i].find_element(By.CLASS_NAME,"yRaY8j").text)
            except:
                d["Original Price"].append(np.nan)
            try:
                d['Discount'].append(elem[i].find_element(By.CLASS_NAME,"UkUFwK").text)
            except:
                d["Discount"].append(np.nan)
            try:
                d['Rating'].append(elem[i].find_element(By.CLASS_NAME,"XQDdHH").text)
            except:
                d["Rating"].append(np.nan)
            try:
                d['Rating & reviews'].append(elem[i].find_element(By.CLASS_NAME, "Wphh3N").text)
            except:
                d["Rating & reviews"].append(np.nan)
            
    except Exception as e:
        print("Error loading cards:", e)

In [17]:
def full_scrape(c,s):
    i=1
    while True:

        url = f"https://www.flipkart.com/search?q={s}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={i}"
        try:
            driver.get(url)
            if len(driver.find_elements(By.XPATH, "//div[@class='slAVV4']")) > 0: 
                scrap_current_page(c,s)
                i+=1
            else:
                print(f"Terminated") 
                break
        except Exception as e:
            print("No more pages or error in pagination:", e)
            break

In [18]:
for c in categories:
    for s in categories[c]:
        search = driver.find_element(By.XPATH, "//input[@placeholder='Search for Products, Brands and More']")
        search.send_keys(s)
        search.send_keys(Keys.RETURN)
        time.sleep(2)
        full_scrape(c,s)
        driver.get(url)

KeyboardInterrupt: 

In [ ]:
import pandas as pd
df = pd.DataFrame(d)
df

,Product,Price,Original Price,Discount,Rating,Rating & reviews,Category,Sub category
0,DCS Pro FITNESS GYM KIT + 3Ft. (Straight + Ben...,"₹1,499","₹3,999",62% off,3.9,(15),"Sports, Fitness, and Outdoor",Sports Equipment
1,"Shopeleven Multi-Functional Training Band, Bod...","₹1,399","₹2,999",53% off,NaN,NaN,"Sports, Fitness, and Outdoor",Sports Equipment
2,Oddish fitness combo Double spring Tummy Trimm...,₹329,"₹1,999",83% off,4,"(3,527)","Sports, Fitness, and Outdoor",Sports Equipment
3,Proactive Sports & Fitness Wooden Rope with Ro...,₹188,₹499,62% off,3.8,(825),"Sports, Fitness, and Outdoor",Sports Equipment
4,Reverie Retail GripGuard Spectacle Ear Hooks: ...,₹190,₹300,36% off,3.6,(25),"Sports, Fitness, and Outdoor",Sports Equipment
...,...,...,...,...,...,...,...,...
5315,BOLDFIT Ankle Binder For Pain Relief Grip Gym ...,₹299,₹999,70% off,3.8,(701),"Sports, Fitness, and Outdoor",Yoga Accessories
5316,BOLDFIT Ankle Support For Pain Relief Ankle Wr...,₹349,₹999,65% off,4.3,"(1,360)","Sports, Fitness, and Outdoor",Yoga Accessories
5317,AGEasy Comfort Knee Cap Anti-Slip with Cotton ...,₹489,"₹1,200",59% off,4,(47),"Sports, Fitness, and Outdoor",Yoga Accessories
5318,Nyamah sales No-Bounce Breast Support Belt Pos...,₹219,₹499,56% off,3.7,"(1,032)","Sports, Fitness, and Outdoor",Yoga Accessories


In [ ]:
df.isnull().sum()

Product                0
Price                  5
Original Price       110
Discount             123
Rating              1814
Rating & reviews    1814
Category               0
Sub category           0
dtype: int64

In [ ]:
df['brand'] = df['Product'].str.split().str[0]
df

,Product,Price,Original Price,Discount,Rating,Rating & reviews,Category,Sub category,brand
0,DCS Pro FITNESS GYM KIT + 3Ft. (Straight + Ben...,"₹1,499","₹3,999",62% off,3.9,(15),"Sports, Fitness, and Outdoor",Sports Equipment,DCS
1,"Shopeleven Multi-Functional Training Band, Bod...","₹1,399","₹2,999",53% off,NaN,NaN,"Sports, Fitness, and Outdoor",Sports Equipment,Shopeleven
2,Oddish fitness combo Double spring Tummy Trimm...,₹329,"₹1,999",83% off,4,"(3,527)","Sports, Fitness, and Outdoor",Sports Equipment,Oddish
3,Proactive Sports & Fitness Wooden Rope with Ro...,₹188,₹499,62% off,3.8,(825),"Sports, Fitness, and Outdoor",Sports Equipment,Proactive
4,Reverie Retail GripGuard Spectacle Ear Hooks: ...,₹190,₹300,36% off,3.6,(25),"Sports, Fitness, and Outdoor",Sports Equipment,Reverie
...,...,...,...,...,...,...,...,...,...
5315,BOLDFIT Ankle Binder For Pain Relief Grip Gym ...,₹299,₹999,70% off,3.8,(701),"Sports, Fitness, and Outdoor",Yoga Accessories,BOLDFIT
5316,BOLDFIT Ankle Support For Pain Relief Ankle Wr...,₹349,₹999,65% off,4.3,"(1,360)","Sports, Fitness, and Outdoor",Yoga Accessories,BOLDFIT
5317,AGEasy Comfort Knee Cap Anti-Slip with Cotton ...,₹489,"₹1,200",59% off,4,(47),"Sports, Fitness, and Outdoor",Yoga Accessories,AGEasy
5318,Nyamah sales No-Bounce Breast Support Belt Pos...,₹219,₹499,56% off,3.7,"(1,032)","Sports, Fitness, and Outdoor",Yoga Accessories,Nyamah


In [19]:
df.to_csv("Flipkart_sabkuch_4.csv")